In [1]:
# Mean Reciprocal Rank

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

if "DATABRICKS_RUNTIME_VERSION" in os.environ and not 'installed_libs' in globals():
  #CUDA = 'cu121' 
  installed_libs = True
  
  
  !pip install torch==2.1.0  torchvision==0.16.0 torchtext==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
  import torch
  #os.environ['TORCH'] = torch.__version__
  #print(torch.__version__)
  #torch_version = '2.0.0+cu118'
  
  #!pip install pyg_lib torch_scatter torch_sparse torch_cluster -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html # torch_spline_conv
  !pip install torch_geometric
  !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html
  #!pip install torch_sparse -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  #!pip install torch_scatter -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  #!pip install pyg_lib -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  !pip install sentence-transformers
  !pip install torcheval
  !pip install matplotlib
  !pip install pandas
  !pip install tensorboard
  
if "DATABRICKS_RUNTIME_VERSION" in os.environ:
  ROOT_FOLDER = '/dbfs/FileStore/GraphNeuralNetworks/'
else:
  ROOT_FOLDER = ''

In [3]:
from learnings_sampler_v1 import get_datasets, uniform_hgt_sampler, get_minibatch_count, add_reverse_edge_original_attributes_and_label_inplace, get_hgt_linkloader, get_single_minibatch_count

train_data, val_data, test_data = get_datasets(get_edge_attr=False, filename=ROOT_FOLDER+'HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt', filter_top_k=True, top_k=50)


/home/amos/mambaforge/envs/pyg_torch21/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


size of dataset on disk:  2.279761238 gb
loading saved heterodata object
for skill job edges keep top k edges per job, k is  50
keep tensor(1208056) of total 16289586


In [5]:
import torch
from models.TransE import TransE
from models.DistMult import DistMult
from models.HGT import HGT
import torch_geometric
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Model(torch.nn.Module):
    def __init__(self, gnn : torch.nn.Module, head :  torch.nn.Module, node_types, edge_types, ggn_output_dim):
        super().__init__()
        # edge_type onehot lookup table with keys
        # node_type onehot lookup table with keys
        self.node_type_embedding = torch.nn.Embedding(len(node_types), ggn_output_dim) # hidden channels should be the output dim of gnn
        
        self.edge_types = edge_types
        for edge_type in edge_types:
            if edge_type[1].startswith('rev_'):
                self.edge_types.remove(edge_type)
        
        # create edge to int mapping
        self.edgeindex_lookup = {edge_type:torch.tensor(i)  for i, edge_type in enumerate(edge_types)}
            
        # hidden channels should be the output dim of gnn
        if head=='TransE': 
            self.head = TransE(len(node_types), len(edge_types) , ggn_output_dim)  # KGE head with loss function
        elif head=='DistMult':
            self.head = DistMult(len(node_types), len(edge_types) , ggn_output_dim)  # KGE head with loss function
        else:
            raise NotImplementedError
        
        self.gnn = gnn
        
    

    def forward(self, hetero_data1, target_edge_type, edge_label_index, edge_label, hetero_data2=None, get_head_fn='loss'):
        
        if hetero_data2 is not None:
            assert target_edge_type[0] != target_edge_type[2], 'when passing two data objects, the edge type has to contain two different node types'
            head_embeddings = self.gnn(hetero_data1.x_dict, hetero_data1.edge_index_dict)[target_edge_type[0]][edge_label_index[0,:]]
            tail_embeddings = self.gnn(hetero_data2.x_dict, hetero_data2.edge_index_dict)[target_edge_type[2]][edge_label_index[1,:]]
        else:
            assert target_edge_type[0] == target_edge_type[2], 'when passing one data object, the edge type has to contain the same node types'


            embeddings = self.gnn(hetero_data1.x_dict, hetero_data1.edge_index_dict)
            head_embeddings = embeddings[target_edge_type[0]][edge_label_index[0,:]]
            tail_embeddings = embeddings[target_edge_type[2]][edge_label_index[1,:]]

        
        edgeindex = self.edgeindex_lookup[target_edge_type]
        if get_head_fn=='loss':
            loss = self.head.loss(head_embeddings, edgeindex.to(device), tail_embeddings, edge_label)
            return loss
        elif get_head_fn=='forward':
            return self.head.forward(head_embeddings, edgeindex.to(device), tail_embeddings)
    
    def get_embeddings(self, hetero_data, rel_type, node_type, first_n_nodes=None):
        # gives us the closeness of embeddings to the other side of the relationship, i.e. e + r or e - r
        if node_type == rel_type[0]:
            have_head_or_tail =='head'
        else:
            have_head_or_tail =='tail'
            
        mask = torch.zeros(hetero_data.x_dict[rel_type[0]].shape[0])
        mask[:first_n_nodes] = 1
        embeddings = self.gnn(hetero_data.x_dict, hetero_data.edge_index_dict)[node_type][mask]
        return self.head.get_embeddings(embeddings, rel_type, have_head_or_tail)
        
        
metadata = train_data.metadata()
# add selfloops
for node_type in train_data.node_types:
    metadata[1].append((node_type, 'self_loop', node_type))    
    
# out_channels = 256
# hidden_channels = 256
# num_heads = 8
# num_layers = 2
# head = 'TransE'



In [6]:
#
model_folder = ROOT_FOLDER+'models/learningpeople_hgt_20231102_004035_lr2e-06_bs32_neighbors_133_1333_head_TransE_hiddenchannels_256_outchannels_256_numheads_8_numlayers_2/Ep0_model_samplesseen154400.pt'

# infer model parameters, dimensions batchsize etc from filename
out_channels = int(model_folder.split('outchannels_')[1].split('_')[0])
hidden_channels = int(model_folder.split('hiddenchannels_')[1].split('_')[0])
num_heads = int(model_folder.split('numheads_')[1].split('_')[0])
num_layers = int(model_folder.split('numlayers_')[1].split('/')[0])
head = model_folder.split('head_')[1].split('_')[0]
learning_rate = float(model_folder.split('lr')[1].split('_')[0])
batch_size = int(model_folder.split('bs')[1].split('_')[0])
num_neighbors = [int(x) for x in model_folder.split('neighbors_')[1].split('head')[0].strip('_').split('_')]


In [7]:
gnn = HGT(hidden_channels=out_channels, out_channels=out_channels, num_heads=num_heads, num_layers=num_layers, node_types=train_data.node_types, data_metadata=metadata)

model = Model(gnn, head=head, node_types=metadata[0], edge_types=metadata[1], ggn_output_dim=out_channels)
#torch_geometric.compile(model, dynamic=True)
#model.load_state_dict(torch.load(model_folder)['model_state_dict'])
model.to(device)

Model(
  (node_type_embedding): Embedding(6, 256)
  (head): TransE(6, num_relations=22, hidden_channels=256)
  (gnn): HGT(
    (lin_dict): ModuleDict(
      (courses_and_programs): Linear(-1, 256, bias=True)
      (qualifications): Linear(-1, 256, bias=True)
      (skills): Linear(-1, 256, bias=True)
      (people): Linear(-1, 256, bias=True)
      (jobs): Linear(-1, 256, bias=True)
      (organizations): Linear(-1, 256, bias=True)
    )
    (convs): ModuleList(
      (0-1): 2 x HGTConv(-1, 256, heads=8)
    )
    (lin): Linear(256, 256, bias=True)
  )
)

In [9]:
input_edgetype = ('people', 'rev_course_and_programs_student', 'courses_and_programs')
add_reverse_edge_original_attributes_and_label_inplace(val_data['courses_and_programs', 'course_and_programs_student', 'people'], reverse_edge=val_data[input_edgetype] )

{'edge_index': tensor([[ 33929,  15584,  28839,  ...,  25609,  28551,  25046],
        [237571, 232648, 223362,  ..., 230612, 245078, 150101]]), 'edge_label': tensor([1., 1., 1.,  ..., 0., 0., 0.]), 'edge_label_index': tensor([[198449,  91945, 160463,  ..., 119261, 257423, 235357],
        [ 19264,  50546,  18282,  ...,   8743,  39182,   1565]])}

In [ ]:
# init dimensions of model by training it
from tqdm.auto import tqdm
from datetime import datetime

input_edgetype = ('people', 'rev_course_and_programs_student', 'courses_and_programs')
add_reverse_edge_original_attributes_and_label_inplace(train_data['courses_and_programs', 'course_and_programs_student', 'people'], reverse_edge=train_data[input_edgetype] )
add_reverse_edge_original_attributes_and_label_inplace(val_data['courses_and_programs', 'course_and_programs_student', 'people'], reverse_edge=val_data[input_edgetype] )


val_sampler = get_hgt_linkloader(val_data, input_edgetype, 4, False, 'triplet', 1, num_neighbors, num_workers=0, prefetch_factor=None, pin_memory=True)

learning_rate = 2e-4
# torch get optimizer by string name
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #2e-15

# train model

model.eval()
minibatchpart1, minibatchpart2, edge_label_index, edge_label, input_edge_id, src_nodes, dst_nodes = next(iter(val_sampler))
#optimizer.zero_grad()
loss = model(minibatchpart1.to(device), input_edgetype, edge_label_index.to(device), edge_label.to(device), minibatchpart2.to(device))
#loss.backward()






In [ ]:
# load state dict
model.load_state_dict(torch.load(model_folder)['model_state_dict'])

In [ ]:
# evaluate model on test set
# init dimensions of model by training it
from tqdm.auto import tqdm
from datetime import datetime
import numpy as np 

mrrs = []
input_edgetype = ('people', 'rev_course_and_programs_student', 'courses_and_programs')
add_reverse_edge_original_attributes_and_label_inplace(test_data['courses_and_programs', 'course_and_programs_student', 'people'], reverse_edge=test_data[input_edgetype] )
negatives = 10000
test_sampler = get_hgt_linkloader(test_data, input_edgetype, 1, False, 'triplet', negatives, num_neighbors, num_workers=0, prefetch_factor=None, pin_memory=True)
best_mrr, best_differences, best_src_nodes, best_dst_nodes = 0, None, None, None
for i, minibatch in tqdm(enumerate(test_sampler)):
    if i==1:
        break
    model.eval()
    minibatchpart1, minibatchpart2, edge_label_index, edge_label, input_edge_id, global_src_nodes, global_dst_nodes = minibatch
    #optimizer.zero_grad()
    differences = model.forward(minibatchpart1.to(device), input_edgetype, edge_label_index.to(device), edge_label.to(device), minibatchpart2.to(device), get_head_fn='forward')
    #loss.backward()

    # define mrr with differences and labels
    # get rank of positive edge from tensor, positive edge is first in batch

    differences = -1* differences.cpu().detach().numpy()
    edge_label = edge_label.cpu().detach().numpy()
    rank = (differences < differences[0]).sum()

    # reciprocal
    mrr = 1/(rank+1)
    if mrr > best_mrr:
        best_mrr = mrr
        best_differences = differences
        best_src_nodes = global_src_nodes
        best_dst_nodes = global_dst_nodes
        print('new best mrr', best_mrr)
        
    print(mrr)
    mrrs.append(mrr)

print('mean mrr',np.mean(mrrs))

In [ ]:
import numpy as np
print(np.mean(mrrs))

In [ ]:
1/0.003534380551853457 +1

In [ ]:
mmr 1/ (rank+1)

rank + 1 = 1/mmr


In [ ]:
edge_label

In [ ]:
train_data_text, val_data_text, test_data_text = get_datasets(get_edge_attr=False, filename=ROOT_FOLDER+'HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt', filter_top_k=True, top_k=50, remove_text_attr=False)
input_edgetype = ('people', 'rev_course_and_programs_student', 'courses_and_programs')
add_reverse_edge_original_attributes_and_label_inplace(test_data_text['courses_and_programs', 'course_and_programs_student', 'people'], reverse_edge=test_data_text[input_edgetype] )



In [ ]:
test_data_text